# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [17]:
import logging
import os
import joblib
import pandas as pd

import azureml.core
from azureml.core import Dataset
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from sklearn.model_selection import train_test_split

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.38.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print("Workspace: " + ws.name, "Region: " + ws.location, sep = '\n')

# choose a name for experiment
experiment_name = 'automl-classification-experiment'

experiment=Experiment(ws, experiment_name)


Workspace: quick-starts-ws-187626
Region: southcentralus


In [3]:
fileHeartFailure = 'data/heart_failure_clinical_records_dataset.csv'
df = pd.read_csv(fileHeartFailure, encoding='latin')
df.head(5)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [4]:
# Default datastore
default_store = ws.get_default_datastore() 

default_store.upload_files([fileHeartFailure], 
                           target_path = 'data', 
                           overwrite = True, 
                           show_progress = True)

heart_failure_data = Dataset.Tabular.from_delimited_files(default_store.path('data/heart_failure_clinical_records_dataset.csv'))

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading data/heart_failure_clinical_records_dataset.csv
Uploaded data/heart_failure_clinical_records_dataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [5]:
key = 'heart_failure_data'
description_text = 'Heart Failure Clinical Records Dataset from Kaggle'

data_set = heart_failure_data.register(workspace=ws,
                                   name=key,
                                   description=key,)

In [6]:
df = data_set.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [7]:
data_set.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           vm_priority= 'dedicated',
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
project_folder = './automl'

automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=data_set,
                             label_column_name="DEATH_EVENT", 
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-classification-experiment,AutoML_a26b767d-03bd-4304-b1b8-e5edd42d6ab9,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
best_run, fitted_automl_best_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
print('Best Run ID',best_run.id)
best_run
fitted_automl_best_model

Best Run ID AutoML_a26b767d-03bd-4304-b1b8-e5edd42d6ab9_43


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0, reg_lambda=1.875, subsample=0.7, tree_method='hist'))], verbose=False)), ('24', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced', criterion='gini', max_depth=None, max_features='log2', max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.06157894736842105, min_samples_split=0.10368421052631578, min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1, oob_score=True, random_state=None, ver

In [21]:
# Metric in best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)
    
print('\nAccuracy of Best run',best_run_metrics['accuracy'],sep='\n')
print(best_run)

model_name = best_run.properties['model_name']
model_name#Register the model
model = remote_run.register_model(model_name=model_name)
print(model.name, model.id, model.version, sep='\t')

recall_score_macro 0.822605380793882
weighted_accuracy 0.8717291910397507
AUC_macro 0.9233179758520148
recall_score_weighted 0.8526436781609196
f1_score_micro 0.8526436781609196
f1_score_macro 0.8219504128441066
recall_score_micro 0.8526436781609196
average_precision_score_micro 0.9299753867891427
AUC_weighted 0.9233179758520148
average_precision_score_macro 0.9120913081727651
precision_score_micro 0.8526436781609196
accuracy 0.8526436781609196
precision_score_weighted 0.8662518792191207
matthews_correlation 0.6651125415988735
f1_score_weighted 0.8491191800514167
AUC_micro 0.9271893248777909
precision_score_macro 0.8448944805194806
balanced_accuracy 0.822605380793882
average_precision_score_weighted 0.9348106851848842
norm_macro_recall 0.6452107615877638
log_loss 0.3645809617330514
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_a26b767d-03bd-4304-b1b8-e5edd42d6ab9_43/accuracy_table
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_a26b767d-03bd-4304-b1b8-e5edd42d6a

'AutoMLa26b767d043'

In [20]:
#TODO: Save the best model

joblib.dump(fitted_automl_best_model, filename='automl.joblib')

['automl.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
#Register the model
model = remote_run.register_model(model_name=model_name)
print(model.name, model.id, model.version, sep='\t')

AutoMLa26b767d043	AutoMLa26b767d043:1	1


In [27]:
env = best_run.get_environment()

script_file = 'score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

'score.py'

In [29]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file, environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                       memory_gb = 1)

deploy_service_name= 'automl-model-deployment'
service = Model.deploy(ws,deploy_service_name,  [model], inference_config, deployment_config)

service.wait_for_deployment(show_output = True)

scoring_uri = service.scoring_uri

print("State: ",service.state)
print("\nScoring URI: ", scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-27 12:15:00+00:00 Creating Container Registry if not exists..
2022-02-27 12:25:01+00:00 Registering the environment.
2022-02-27 12:25:01+00:00 Use the existing image.
2022-02-27 12:25:01+00:00 Generating deployment configuration.
2022-02-27 12:25:02+00:00 Submitting deployment to compute..
2022-02-27 12:25:06+00:00 Checking the status of deployment automl-model-deployment..
2022-02-27 12:27:56+00:00 Checking the status of inference endpoint automl-model-deployment.
Succeeded
ACI service creation operation finished, operation "Succeeded"
State:  Healthy

Scoring URI:  http://edf94abc-214c-47cd-b4e0-3a314e3698e3.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [30]:
import json
data_sample = df.sample(3)
y_true = data_sample.pop('DEATH_EVENT')
sample_json = json.dumps({'data':data_sample.to_dict(orient='records')})
print(sample_json)
output = service.run(sample_json)
print(output)

{"data": [{"age": 48.0, "anaemia": 1, "creatinine_phosphokinase": 582, "diabetes": 1, "ejection_fraction": 55, "high_blood_pressure": 0, "platelets": 87000.0, "serum_creatinine": 1.9, "serum_sodium": 121, "sex": 0, "smoking": 0, "time": 15}, {"age": 94.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 1, "ejection_fraction": 38, "high_blood_pressure": 1, "platelets": 263358.03, "serum_creatinine": 1.83, "serum_sodium": 134, "sex": 1, "smoking": 0, "time": 27}, {"age": 60.0, "anaemia": 1, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 30, "high_blood_pressure": 1, "platelets": 127000.0, "serum_creatinine": 0.9, "serum_sodium": 145, "sex": 0, "smoking": 0, "time": 95}]}
{"result": [1, 1, 0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [31]:
print("\nService Logs:\n",service.get_logs())


Service Logs:
 2022-02-27T12:27:30,589250000+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-02-27T12:27:30,598215800+00:00 - iot-server/run 
2022-02-27T12:27:30,596315800+00:00 - rsyslog/run 
2022-02-27T12:27:30,750170700+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_6797cf9b513e59b405ce80f3e9222a7d/lib/libuuid.so.1: no version information available (required by rsyslogd)
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (75)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 94
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-02-27T12:27:34,818509000+00:00 - iot-server/finish 1 0
2022-02-27T12:27:34,820788900+00:00 - Exit code 1 is normal. Not restarting iot-server.
Initializing logger
2022-02-27 12:27:35,397 | root | INFO | Starting up app insights client
log

In [32]:
service.delete()
compute_target.shutdown()


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
